In [2]:
import rasterio
from rasterio.windows import Window
import os

# Create a folder named "images" if it doesn't exist
folder_name = "labels"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Open the input raster file
with rasterio.open('mask.tif') as src:

    # Calculate the height and width of the sub-rasters
    height = src.height // 20  # Divide the raster into 4 rows
    width = src.width // 8  # Divide the raster into 8 columns

    # Loop through each sub-raster and clip the data
    for i in range(20):
        for j in range(8):
            # Define the window to clip the sub-raster
            ymin = i * height
            ymax = (i + 1) * height
            xmin = j * width
            xmax = (j + 1) * width
            window = Window.from_slices((ymin, ymax), (xmin, xmax))

            # Read the data within the window
            clipped_data = src.read(window=window)

            # Update the metadata for the clipped data
            clipped_meta = src.meta.copy()
            clipped_meta.update({
                'height': window.height,
                'width': window.width,
                'transform': src.window_transform(window)
            })

            # Write the clipped data to a new raster file
            # with rasterio.open(f'sub_raster_{i}_{j}.tif', 'w', **clipped_meta) as dst:
            #     dst.write(clipped_data)
            output_file = os.path.join(folder_name, f'image_{i}_{j}_mask.png')
            with rasterio.open(output_file, 'w', **clipped_meta) as dst:
                 dst.write(clipped_data)